In [8]:
# default_exp preprocess.prepare_transaction_data

# Preparing transaction data for the customers in the scope 

> API details.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
import os
import pandas as pd
from sample_project import config
from sample_project.helper import write_to_csv, read_from_csv
from fastcore.utils import store_attr
import numpy as np

In [4]:
#hide
import warnings
warnings.filterwarnings("ignore")

In [6]:
#export
class Create_Data:
    '''
    This class is built to create main dataset to be used in this project. Following steps below, the transaction dataset for clients in the scope is created:
    1. Merging three different datasets: transaction, customer info and disp info which shows customer and account matches
    2. Applying three different filters below to cover only customers in the scope:
        - Consider only transactions whose date in between <start date> and <end date> 
        - Consider only customers who have loans with more than <loan_amnt_thrsh> euros
        - Consider only customers from districts which have more than <district_cnt_thrsh> customers 
        
    Args:
            trnx_dataset (Pandas DataFrame): The csv file name which has transaction dataset with at least these fields: "account_id","date"
            disp_dataset (Pandas DataFrame): The csv file name which disp dataset with at least these fields: "client_id","account_id"
            client_dataset (Pandas DataFrame): The csv file name which customer info dataset with at least these fields: "client_id","district_id"
            loan_dataset (Pandas DataFrame): The csv file name which loan dataset with at least these fields: "client_id","amount"
            loan_amnt_thrsh (integer): Loan amount threshold to be use to apply filter 2
            district_cnt_thrsh (integer): District count threshold to be used to apply filter 3
            start_date (integer): Start date threshold for transaction dataset to apply filter 1
            end_date (integer): End date threshold for transaction dataset to apply filter 1
            to_csv (boolean): If the returned dataframe is desired to be written into csv file 

        Return:
            main_data (pandas DataFrame): the transaction dataset for clients in the scope 
    '''
    def __init__(self,trnx_dataset=None, disp_dataset=None, client_dataset=None, loan_dataset=None, loan_amnt_thrsh=1000, 
                 district_cnt_thrsh = 100, start_date=900000, end_date=1000000, to_csv=True):

        store_attr()
        
        if trnx_dataset == None: trnx_dataset = config.CSV_TRANSACTION
        self.trnx_dataset = read_from_csv(trnx_dataset)
        
        if disp_dataset == None: disp_dataset = config.CSV_DISP_INFO
        self.disp_dataset = read_from_csv(disp_dataset)
        
        if client_dataset == None: client_dataset = config.CSV_CUST_INFO
        self.client_dataset = read_from_csv(client_dataset)
            
        if loan_dataset == None: loan_dataset = config.CSV_LOAN
        self.loan_dataset = read_from_csv(loan_dataset)
        
    
    def __call__(self):
        
        main_data = self._applying_date_filter(self._merge_main_datasets())
        main_data = self._applying_loan_amount_filter(main_data)
        main_data = self._applying_district_filter(main_data)
        
        if self.to_csv:
            
            write_to_csv(df= main_data, path = config.CSV_CUSTOMIZED_TRNX )
            
        return main_data
         
    def _merge_main_datasets(self):
        
        return  (self.trnx_dataset.drop(["k_symbol","bank","account"],axis=1)
                 .merge(self.disp_dataset[["client_id","account_id"]], on="account_id",how="left")
                 .merge(self.client_dataset[["client_id","district_id"]],on="client_id",how="left")
                ) 
        
    def _applying_date_filter(self,df):
        
        return df[(df["date"] >= self.start_date)&(df["date"] <= self.end_date)]
    
    def _applying_loan_amount_filter(self,df):
        
        df = df.merge(
                        (self.loan_dataset
                             .merge(self.disp_dataset[["client_id","account_id"]],on="account_id",how="left")
                             .groupby("client_id",as_index=False).amount.sum().rename(columns={'amount':'Total_Loan_Amount'})
                         ), on ="client_id", how="left"
                      )
        df["Total_Loan_Amount"] = df["Total_Loan_Amount"].fillna(0)
        
        return df[df["Total_Loan_Amount"] > self.loan_amnt_thrsh]
    
    def _applying_district_filter(self,df):
        
        district_count = self.client_dataset.groupby("district_id",as_index=False).client_id.count().rename(columns={'client_id':'Num_Cust_in_District'})
        district_list = district_count[district_count["Num_Cust_in_District"]>self.district_cnt_thrsh]["district_id"].tolist()
        
        return df[df["district_id"].isin(district_list)]
    

In [7]:
#hide
data = Create_Data(loan_amnt_thrsh=0, district_cnt_thrsh = 0, to_csv=True)
data()

,trans_id,account_id,date,type,operation,amount,balance,client_id,district_id,Total_Loan_Amount
81,1548749,5270,930113,"""PRIJEM""","""VKLAD""",800.0,800.0,6367,44,79608.0
91,1548750,5270,930114,"""PRIJEM""","""PREVOD Z UCTU""",44749.0,45549.0,6367,44,79608.0
94,3393738,11265,930114,"""PRIJEM""","""VKLAD""",1000.0,1000.0,13845,15,52788.0
100,3122924,10364,930117,"""PRIJEM""","""VKLAD""",1100.0,1100.0,12754,55,21924.0
101,3122924,10364,930117,"""PRIJEM""","""VKLAD""",1100.0,1100.0,12755,55,21924.0
...,...,...,...,...,...,...,...,...,...,...
1262579,3626751,2910,981231,"""PRIJEM""","""""",187.2,42249.6,3517,68,437460.0
1262585,3627577,2933,981231,"""PRIJEM""","""""",132.0,31046.4,3542,10,272520.0
1262586,3627577,2933,981231,"""PRIJEM""","""""",132.0,31046.4,3543,10,272520.0
1262597,3626847,2912,981231,"""PRIJEM""","""""",271.6,58694.2,3519,74,380160.0
